![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite.png)
# Isochrone Generation Example
Isochrones are a type of _iso line_—a contour that connects points sharing the same value. While contour lines link points of equal elevation and isotherms connect equal temperatures, isochrones outline regions that are reachable within a specific amount of travel time. They're commonly used to answer questions like: How far can I drive from here in 15 minutes? or Which neighborhoods are within a 10-minute walk of a school?

In this example we will generate `10`, `20`, and `30` minute isochrones for all the fire stations in California. Then we will left join the California places overture table to those isochrones to label them as being within 10/20/30/>30 minutes from a fire station. Finally we will generate clusters with DBSCAN to determine if there are regions where a firestation might be useful

## Define Sedona Context

In [2]:
from sedona.spark import *
from  wherobots.sql.st_functions import ST_Isochrones
import pyspark.sql.functions as f
import os

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Initialize Data

Lets get all the places from Overture in California. Then we will also filter down to just those that are fire departments.

In [6]:
ca_polygon = "POLYGON((-124.4009 41.9983,-123.6237 42.0024,-123.1526 42.0126,-122.0073 42.0075,-121.2369 41.9962,-119.9982 41.9983,-120.0037 39.0021,-117.9575 37.5555,-116.3699 36.3594,-114.6368 35.0075,-114.6382 34.9659,-114.6286 34.9107,-114.6382 34.8758,-114.5970 34.8454,-114.5682 34.7890,-114.4968 34.7269,-114.4501 34.6648,-114.4597 34.6581,-114.4322 34.5869,-114.3787 34.5235,-114.3869 34.4601,-114.3361 34.4500,-114.3031 34.4375,-114.2674 34.4024,-114.1864 34.3559,-114.1383 34.3049,-114.1315 34.2561,-114.1651 34.2595,-114.2249 34.2044,-114.2221 34.1914,-114.2908 34.1720,-114.3237 34.1368,-114.3622 34.1186,-114.4089 34.1118,-114.4363 34.0856,-114.4336 34.0276,-114.4652 34.0117,-114.5119 33.9582,-114.5366 33.9308,-114.5091 33.9058,-114.5256 33.8613,-114.5215 33.8248,-114.5050 33.7597,-114.4940 33.7083,-114.5284 33.6832,-114.5242 33.6363,-114.5393 33.5895,-114.5242 33.5528,-114.5586 33.5311,-114.5778 33.5070,-114.6245 33.4418,-114.6506 33.4142,-114.7055 33.4039,-114.6973 33.3546,-114.7302 33.3041,-114.7206 33.2858,-114.6808 33.2754,-114.6698 33.2582,-114.6904 33.2467,-114.6794 33.1720,-114.7083 33.0904,-114.6918 33.0858,-114.6629 33.0328,-114.6451 33.0501,-114.6286 33.0305,-114.5888 33.0282,-114.5750 33.0351,-114.5174 33.0328,-114.4913 32.9718,-114.4775 32.9764,-114.4844 32.9372,-114.4679 32.8427,-114.5091 32.8161,-114.5311 32.7850,-114.5284 32.7573,-114.5641 32.7503,-114.6162 32.7353,-114.6986 32.7480,-114.7220 32.7191,-115.1944 32.6868,-117.3395 32.5121,-117.4823 32.7838,-117.5977 33.0501,-117.6814 33.2341,-118.0591 33.4578,-118.6290 33.5403,-118.7073 33.7928,-119.3706 33.9582,-120.0050 34.1925,-120.7164 34.2561,-120.9128 34.5360,-120.8427 34.9749,-121.1325 35.2131,-121.3220 35.5255,-121.8013 35.9691,-122.1446 36.2808,-122.1721 36.7268,-122.6871 37.2227,-122.8903 37.7783,-123.2378 37.8965,-123.3202 38.3449,-123.8338 38.7423,-123.9793 38.9946,-124.0329 39.3088,-124.0823 39.7642,-124.5314 40.1663,-124.6509 40.4658,-124.3144 41.0110,-124.3419 41.2386,-124.4545 41.7170,-124.4009 41.9983,-124.4009 41.9983))"
ca_places_df = sedona.table("wherobots_open_data.overture_2025_01_22_0.places_place").where(f"ST_Intersects(geometry, ST_GeomFromText('{ca_polygon}'))")\
    .repartition(144 * 2) # Havasu will only hit a few files; lets make sure our dataframe has enough partitions to utilize our cluster well.
fires_df = ca_places_df.where(f"categories.primary = 'fire_department'")
fires_df.count()

1857

## Generate the isochrones

Here we generate isochrones for `10`, `20`, and `30` minutes for all 1,857 fire stations in out dataframe.

There are 4 arguments for the `ST_Isochrones()` funcion:

1. `geometry`: The starting point from which we will generate the isochrone.
2. `time_limits`: An array of travel times in minutes; In our case `10`, `20`, and `30`.
3. `mobility_type`: Transportation; In our case, `car`.
4. `inbound`: Indicates if the geometry is a destination (true) or origin (false); in our case the points represent the `origin`.
5. `isolate_contours`: If true, creates concentric rings; if false, creates overlapping isochrones; we are creating overlapping isochrones.


You'll notice this dataframe is marked for caching. Caching allows the database to plan the join efficiently without calculating the isochrones twice.

In the result below, you can see that the isochrones are returned as a list of polygons with the order matching the time limits. 

In [9]:
%%time
isochrones_df = fires_df.withColumn(
    "isochrones",
    ST_Isochrones(f.col("geometry"), 
                  f.array(f.lit(10), f.lit(20), f.lit(30)),
                  f.lit("car"), 
                  f.lit(False), 
                  f.lit(False))
).cache()

isochrones_df.select("id","names","isochrones").show()

25/03/25 20:38:56 WARN CacheManager: Asked to cache already cached data.


+--------------------+--------------------+--------------------+
|                  id|               names|          isochrones|
+--------------------+--------------------+--------------------+
|08f29ad13190b9660...|{CAL FIRE/South B...|[POLYGON ((-120.8...|
|08f29add4b4e86de0...|{CAL FIRE San Lui...|[POLYGON ((-120.7...|
|08f29ad024d23d600...|{CAL FIRE San Lui...|[POLYGON ((-120.9...|
|08f29add5d8589250...|{CAL FIRE/Pismo B...|[POLYGON ((-120.7...|
|08f29adc461a1b200...|{San Luis Obispo ...|[POLYGON ((-120.7...|
|08f29adc43ced0000...|{The California D...|[POLYGON ((-120.7...|
|08f29adc701416c60...|{Airport Fire Sta...|[POLYGON ((-120.6...|
|08f29ad1989a93b10...|{Atascadero Fire ...|[POLYGON ((-120.7...|
|08f29adc461127540...|{Station 1, NULL,...|[POLYGON ((-120.7...|
|08f29ad06e0133850...|{Cambria Fire Dep...|[POLYGON ((-121.1...|
|08f29ad70c4d04440...|{Camp Roberts Fir...|[POLYGON ((-120.8...|
|08f29ad0840e1b800...|{CAL FIRE San Lui...|[POLYGON ((-120.7...|
|08f29ad083574b6a0...|{Pa

#### Map the Results with Sedona Kepler

In [ ]:
SedonaKepler.create_map(isochrones_df.withColumn("geometry", f.col("isochrones")[0]),name="Isochrones")

## How far away is the closest Fire Station to each Overture Maps Fondation _Places_ record?
Here we label each of the locations in the Overture Maps Foundation `Places` data set with the fire station isochrone bucket that it falls within. THis is donw by performing a left join the places table to the isochrones and then determining the smallest isochrone they intersect.

In [15]:
%%time

risk_df = (
    ca_places_df.alias("ca")
    .join(isochrones_df.alias("isochrones"), ST_Intersects(isochrones_df.isochrones[2], "ca.geometry"), "left")
    .withColumn(
        "bucket",
        f.when(isochrones_df.isochrones.isNull(), f.lit(None))
        .when(ST_Intersects(isochrones_df.isochrones[0], "ca.geometry"), 10)
        .when(ST_Intersects(isochrones_df.isochrones[1], "ca.geometry"), 20)
        .when(ST_Intersects(isochrones_df.isochrones[2], "ca.geometry"), 30)
    )
    .groupBy(f.col("ca.id")) 
    .agg(*[f.first("ca." + field).alias(field) for field in ca_places_df.columns if field != "id"], f.min(f.col("bucket")).alias("bucket"))
    .cache()
)

risk_df.write.format("geoparquet").mode("overwrite").save(os.getenv("USER_S3_PATH") + "fireRiskExampleIsochrones")
isochrones_df.unpersist()

risk_df.select("names","geometry","bucket").show()

25/03/25 20:52:57 WARN CacheManager: Asked to cache already cached data.


+--------------------+--------------------+------+
|               names|            geometry|bucket|
+--------------------+--------------------+------+
|{Six Rivers Natio...|POINT (-123.62568...|    20|
|{Top Of Trinidad ...|POINT (-124.15022...|    10|
|{Pearson's Grocer...|POINT (-123.70507...|    20|
|{B&B Portable Toi...|POINT (-123.99352...|    10|
|{Mad River Hammon...|POINT (-124.12040...|    10|
|{Lane Grove, NULL...|POINT (-123.78989...|    10|
|{Auto Expo, NULL,...|POINT (-124.14636...|    10|
|{Fernbridge, NULL...|POINT (-124.20247...|    10|
|{County of Humbol...|POINT (-124.14532...|    10|
|{Eureka Massage a...|POINT (-124.13651...|    10|
|{Manila Park, NUL...|POINT (-124.16153...|    10|
|{Pauli-Shaw Insur...|POINT (-124.08545...|    10|
|{Providence Medic...|POINT (-124.13719...|    10|
|{Starbucks, NULL,...|POINT (-124.13841...|    10|
|{Central Transpor...|POINT (-124.183 4...|    10|
|{I Am Limitless L...|POINT (-124.17808...|    10|
|{Gaining Ground, ...|POINT (-1

## Find Clusters of OMF Place *not* near Fire Stations
Finally, we use WherobotsDB's DBSCAN to find clsuters of places that are not close to a fire station.

In [ ]:
%%time
from math import pi

R = 6371000
C = 2 * pi * R
METERS_PER_DEGREE = C / 360.0

def degreesFromMeters(distanceInMeters):
    return distanceInMeters / METERS_PER_DEGREE


clustered_high_risk_df = (
    risk_df
    .where(risk_df.bucket > 10)
    .withColumn("dbscan_result", ST_DBSCAN(risk_df.geometry, degreesFromMeters(1000), 50, False))
    .groupBy("dbscan_result.cluster")
    .agg(ST_ConvexHull(ST_Union_Aggr("geometry")).alias("geometry"))
    .where("cluster != -1") # dont show outliers
).cache()

#### Map the Results with Sedona Kepler

In [16]:
SedonaKepler.create_map(clustered_high_risk_df, name="Clusters")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Clusters': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…